In [5]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import pandas as pd
import requests
from io import StringIO
from tqdm import tqdm
from modlamp.descriptors import GlobalDescriptor
import pyarrow as pa


In [50]:
import requests
import pandas as pd
from io import StringIO
from tqdm.auto import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def fetch_uniprot_cursor(query: str = "reviewed:true", total: int = 1000, batch_size=500) -> pd.DataFrame:
    base_url = "https://rest.uniprot.org/uniprotkb/search"
    fields = (
        "accession,id,protein_name,organism_name,length,sequence,mass,"
        "cc_subcellular_location,go_p,go_f,ec,cc_disruption_phenotype,"
        "cc_catalytic_activity,cc_pathway,cc_subcellular_location,"
        "cc_function,cc_domain,cc_induction,cc_pharmaceutical,"
        "cc_disruption_phenotype"
    )

    headers = {"Accept": "text/tab-separated-values"}
    params = {
        "query": query,
        "format": "tsv",
        "fields": fields,
        "size": batch_size,
    }

    # Set up retry strategy
    retry_strategy = Retry(
        total=30,
        backoff_factor=2,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["HEAD", "GET", "OPTIONS"]
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session = requests.Session()
    session.mount("https://", adapter)
    session.mount("http://", adapter)

    results = []
    next_url = base_url
    fetched = 0

    with tqdm(total=total, desc="Fetching UniProt entries") as pbar:
        while next_url and fetched < total:
            try:
                if fetched == 0:
                    resp = session.get(next_url, params=params, headers=headers, timeout=10)
                else:
                    resp = session.get(next_url, headers=headers, timeout=10)
                resp.raise_for_status()
            except requests.exceptions.RequestException as e:
                print(f"Request failed: {e}")
                break

            df = pd.read_csv(StringIO(resp.text), sep="\t")
            if df.empty:
                break

            results.append(df)
            fetched += len(df)
            pbar.update(len(df))

            # get next URL from headers (cursor-based)
            next_url = resp.links.get("next", {}).get("url", None)

    if results:
        return pd.concat(results, ignore_index=True).iloc[:total].reset_index(drop=True)
    else:
        return pd.DataFrame()


In [51]:
import math
import requests
import pandas as pd
from io import StringIO
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm

def fetch_uniprot_batches_parallel(
    query: str = "reviewed:true",
    batch_size: int = 500,
    total: int = 1000,
    max_workers: int = 32 
) -> pd.DataFrame:
    """
    Parallelized fetch of UniProt search results in TSV format.

    Args:
      query: UniProt query string
      batch_size: number of entries per page
      total: total entries desired
      max_workers: number of threads to use for HTTP fetches

    Returns:
      Pandas DataFrame with up to `total` rows.
    """
    base_url = "https://rest.uniprot.org/uniprotkb/search"
    fields = (
        "accession,id,protein_name,organism_name,length,sequence,mass,"
        "cc_subcellular_location,go_p,go_f,ec,cc_disruption_phenotype,"
        "cc_catalytic_activity,cc_pathway,cc_subcellular_location,"
        "cc_function,cc_domain,cc_induction,cc_pharmaceutical,"
        "cc_disruption_phenotype"
    )

    # how many pages we need
    num_pages = math.ceil(total / batch_size)
    
    def fetch_page(offset: int) -> pd.DataFrame:
        params = {
            "query": query,
            "format": "tsv",
            "fields": fields,
            "size": batch_size,
            "offset": offset
        }
        resp = requests.get(base_url, params=params)
        resp.raise_for_status()
        return pd.read_csv(StringIO(resp.text), sep="\t")

    # kick off all page fetches
    offsets = [i * batch_size for i in range(num_pages)]
    dfs = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(fetch_page, off): off for off in offsets}
        for future in tqdm(as_completed(futures),
                           total=len(futures),
                           desc="UniProt pages"):
            df = future.result()
            if df.empty:
                break
            dfs.append(df)

    # combine and cap to `total`
    all_data = pd.concat(dfs, ignore_index=True)
    return all_data.iloc[:total].reset_index(drop=True)


In [52]:
def fetch_uniprot_batches(query="reviewed:true", batch_size=500, total=10000):
    base_url = "https://rest.uniprot.org/uniprotkb/search"
    all_dataframes = []
    fetched = 0
    params = {
        "query": query,
        "format": "tsv",
        "fields": (
            "accession,id,protein_name,organism_name,length,sequence,mass,"
            "cc_subcellular_location,go_p,go_f,ec,cc_disruption_phenotype,"
            "cc_catalytic_activity,cc_pathway,cc_function,cc_domain,"
            "cc_induction,cc_pharmaceutical"
        ),
        "size": batch_size,
    }

    response = requests.get(base_url, params=params)
    if not response.ok:
        raise Exception(f"Initial fetch failed: {response.status_code} {response.text}")

    with tqdm(total=total, desc="Fetching UniProt entries") as pbar:
        while True:
            df_batch = pd.read_csv(StringIO(response.text), sep="\t")
            if df_batch.empty:
                break

            all_dataframes.append(df_batch)
            batch_size = len(df_batch)
            fetched += batch_size
            pbar.update(batch_size)

            if fetched >= total:
                break

          # Get full next URL from response.links
            next_url = response.links.get("next", {}).get("url", None)
            if not next_url:
                break

            response = requests.get(next_url)
            if not response.ok:
                raise Exception(f"Next fetch failed: {response.status_code} {response.text}")

    return pd.concat(all_dataframes, ignore_index=True)



In [53]:
def compute_biopython_features(seq: str, entry: str) -> dict:
    analysis = ProteinAnalysis(seq)

    try:
        aa_counts = analysis.count_amino_acids()
        aa_percents = analysis.get_amino_acids_percent()
        sec_frac = analysis.secondary_structure_fraction()
    except Exception as e:
        print(f"Failed on sequence: {seq} - {e}")
        return None

    try:
        features = {
            "Entry": entry,
            "Sequence": seq,
            "length": len(seq),
            "mol_weight": analysis.molecular_weight(),
            "iso_point": analysis.isoelectric_point(),
            "aromaticity": analysis.aromaticity(),
            "instability_index": analysis.instability_index(),
            "gravy": analysis.gravy(),
            "helix_frac": sec_frac[0],
            "turn_frac": sec_frac[1],
            "sheet_frac": sec_frac[2],
        }
    except Exception as e:
        #print(f"Failed to compute features for sequence: {seq} - {e}")
        return None

    # Add amino acid counts and percents
    for aa in aa_counts:
        features[f"count_{aa}"] = aa_counts[aa]
        features[f"percent_{aa}"] = round(aa_percents[aa],3)

    
    # --- modlamp GlobalDescriptor ---
    try:
        desc = GlobalDescriptor([seq])
        desc.calculate_all()
        modlamp_feats = desc.descriptor[0]
        modlamp_names = [
            "charge_pH7", "boman_index", "aliphatic_index", "hydrophobic_moment"
        ]

        for name, value in zip(modlamp_names, modlamp_feats):
            orig_name = name.split("_", 1)[-1]
            features[name] = value

    except Exception as e:
        #print(f"modlamp failed on sequence: {seq} - {e}")
        return None

    return features


In [54]:
from concurrent.futures import ProcessPoolExecutor

def parallel_compute_features(seq_entry_list, max_workers=None):
    """
    seq_entry_list: List of (seq, entry) tuples
    max_workers:    Number of processes to spin up (defaults to os.cpu_count())
    """
    results = []
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        # schedule all the jobs
        futures = {
            executor.submit(compute_biopython_features, seq, entry): (seq, entry)
            for seq, entry in seq_entry_list
        }

        # progress over completions
        for future in tqdm(as_completed(futures),
                           total=len(futures),
                           desc="Computing features"):
            seq, entry = futures[future]
            try:
                feat = future.result()
                if feat is not None:
                    results.append(feat)
            except Exception as exc:
                print(f"[Worker error] seq={entry} raised {exc!r}")
    return results

In [56]:
df = fetch_uniprot_cursor(batch_size=500, total= 1000000)
df_filtered = df[df['Length'] <= 1024]
df_filtered = df = df.drop(columns=['Pharmaceutical use'])

Fetching UniProt entries:  57%|█████▋    | 573230/1000000 [31:27<23:25, 303.64it/s]  


In [57]:
df_filtered.nunique()


Entry                                 573230
Entry Name                            573230
Protein names                         169016
Organism                               14778
Length                                  3476
Sequence                              484998
Mass                                  110205
Subcellular location [CC]              59530
Gene Ontology (biological process)     66513
Gene Ontology (molecular function)     40407
EC number                               7141
Disruption phenotype                   19948
Catalytic activity                     30481
Pathway                                 8622
Subcellular location [CC].1            59530
Function [CC]                         115427
Domain [CC]                            13940
Induction                              16973
Disruption phenotype.1                 19948
dtype: int64

In [58]:
sequences = df_filtered["Sequence"].tolist()
entry = df_filtered["Entry"].tolist()
seq_entry_list = list(zip(sequences, entry))

feature_dicts = []
feature_dicts = parallel_compute_features(seq_entry_list, max_workers=64)

/home/ubuntu/.local/lib/python3.10/site-packages/Bio/SeqUtils/ProtParam.py:106: BiopythonDeprecationWarning: The get_amino_acids_percent method has been deprecated and will likely be removed from Biopython in the near future. Please use the amino_acids_percent attribute instead.
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/Bio/SeqUtils/ProtParam.py:106: BiopythonDeprecationWarning: The get_amino_acids_percent method has been deprecated and will likely be removed from Biopython in the near future. Please use the amino_acids_percent attribute instead.
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/Bio/SeqUtils/ProtParam.py:106: BiopythonDeprecationWarning: The get_amino_acids_percent method has been deprecated and will likely be removed from Biopython in the near future. Please use the amino_acids_percent attribute instead.
  warnings.warn(
/home/ubuntu/.local/lib/python3.10/site-packages/Bio/SeqUtils/ProtParam.py:106: BiopythonDeprecationWarning: 

In [59]:
len(feature_dicts)

570558

In [67]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm

output_path = "datatest573230.parquet"
chunk_size = 5000
writer = None
schema = None
reference_columns_and_types = None

def fix_null_columns(df, reference_columns_and_types):
    for col, dtype in reference_columns_and_types.items():
        if col not in df.columns:
            df[col] = pd.NA
        try:
            df[col] = df[col].astype(dtype)
        except Exception:
            df[col] = df[col].astype("object")
    return df

def sanitize_for_arrow(df):
    for col in df.columns:
        if df[col].dtype == object:
            df[col] = df[col].astype(str)  # Coerce to string
    return df

for i in tqdm(range(0, len(feature_dicts), chunk_size)):
    chunk_df = pd.DataFrame(feature_dicts[i:i + chunk_size])

    if reference_columns_and_types is None:
        reference_columns_and_types = {
            col: str(chunk_df[col].dtype) for col in chunk_df.columns
        }

    chunk_df = fix_null_columns(chunk_df, reference_columns_and_types)
    merged = chunk_df.merge(df, on="Sequence", how="left")

    if len(merged) > 2 * len(chunk_df):
        print(len(merged))
        print(len(chunk_df))
        print("Merge resulted in too many rows. Check for duplicates.")

    merged = sanitize_for_arrow(merged)

    table = pa.Table.from_pandas(merged, preserve_index=False)

    if writer is None:
        schema = pa.schema([
            pa.field(name, typ).with_nullable(True)
            for name, typ in zip(table.schema.names, table.schema.types)
        ])
        writer = pq.ParquetWriter(output_path, schema, compression="snappy")

    table = pa.Table.from_pandas(merged, schema=schema, preserve_index=False)
    writer.write_table(table)

if writer is not None:
    writer.close()


  0%|          | 0/115 [00:00<?, ?it/s]

 23%|██▎       | 27/115 [00:19<01:04,  1.37it/s]

10801
5000
Merge resulted in too many rows. Check for duplicates.


 24%|██▍       | 28/115 [00:20<01:05,  1.34it/s]

12678
5000
Merge resulted in too many rows. Check for duplicates.


...

 98%|█████████▊| 113/115 [01:27<00:01,  1.30it/s]

12132
5000
Merge resulted in too many rows. Check for duplicates.


 99%|█████████▉| 114/115 [01:28<00:00,  1.30it/s]

13494
5000
Merge resulted in too many rows. Check for duplicates.


100%|██████████| 115/115 [01:29<00:00,  1.29it/s]


In [68]:
check = pd.read_parquet("datatest573230.parquet", engine="pyarrow")


In [69]:
check.nunique()


Entry_x                        570558
Sequence                       482356
length                           3464
mol_weight                     480443
iso_point                      101191
                                ...  
Subcellular location [CC].1     59332
Function [CC]                  114896
Domain [CC]                     13902
Induction                       16931
Disruption phenotype.1          19927
Length: 73, dtype: int64

In [72]:
checkdropped = check.drop_duplicates(subset=['Sequence']).reset_index(drop=True)


In [ ]:
len(checkdropped)


482356

: 